In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [2]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [3]:
Y_train

array([9, 8, 2, ..., 5, 7, 8])

In [4]:
def init_params():
    W1 = np.random.normal(size=(128, 784)) * np.sqrt(1./(784))
    b1 = np.random.normal(size=(128, 1)) * np.sqrt(1./128)
    W2 = np.random.normal(size=(64, 128)) * np.sqrt(1./(128))
    b2 = np.random.normal(size=(64, 1)) * np.sqrt(1./64)
    W3 = np.random.normal(size=(10, 64)) * np.sqrt(1./(64))
    b3 = np.random.normal(size=(10, 1)) * np.sqrt(1./10)
    return W1, b1, W2, b2, W3, b3


def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A


def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3


def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    one_hot_Y = one_hot(Y)
    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3, axis=1, keepdims=True)
    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2, dW3, db3


def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1    
    W2 -= alpha * dW2  
    b2 -= alpha * db2
    W3 -= alpha * dW3
    b3 -= alpha * db3
    return W1, b1, W2, b2, W3, b3


def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A3)
            print("Accuracy: ", get_accuracy(predictions, Y))
    return W1, b1, W2, b2, W3, b3

W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, 0.10, 500)



Iteration:  0
[0 0 0 ... 4 2 2] [9 8 2 ... 5 7 8]
Accuracy:  0.10321951219512195
Iteration:  10
[4 1 0 ... 3 7 2] [9 8 2 ... 5 7 8]
Accuracy:  0.48841463414634145
Iteration:  20
[1 8 2 ... 3 7 8] [9 8 2 ... 5 7 8]
Accuracy:  0.6820975609756098
Iteration:  30
[9 8 2 ... 3 7 8] [9 8 2 ... 5 7 8]
Accuracy:  0.762219512195122
Iteration:  40
[9 8 2 ... 3 7 8] [9 8 2 ... 5 7 8]
Accuracy:  0.8089268292682927
Iteration:  50
[9 8 2 ... 5 7 8] [9 8 2 ... 5 7 8]
Accuracy:  0.8357073170731707
Iteration:  60
[9 8 2 ... 5 7 8] [9 8 2 ... 5 7 8]
Accuracy:  0.8521951219512195
Iteration:  70
[9 8 2 ... 5 7 8] [9 8 2 ... 5 7 8]
Accuracy:  0.8641219512195122
Iteration:  80
[9 8 2 ... 5 7 8] [9 8 2 ... 5 7 8]
Accuracy:  0.8709512195121951
Iteration:  90
[9 8 2 ... 5 7 8] [9 8 2 ... 5 7 8]
Accuracy:  0.8771951219512195
Iteration:  100
[9 8 2 ... 5 7 8] [9 8 2 ... 5 7 8]
Accuracy:  0.8816341463414634
Iteration:  110
[9 8 2 ... 5 7 8] [9 8 2 ... 5 7 8]
Accuracy:  0.8849024390243903
Iteration:  120
[9 8 2 ...

In [5]:
def make_predictions(X, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)  
    predictions = get_predictions(A3)
    return predictions

def test_prediction(index, W1, b1, W2, b2, W3, b3):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2, W3, b3)  
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()
    
    test_prediction(0, W1, b1, W2, b2, W3, b3)
    test_prediction(1, W1, b1, W2, b2, W3, b3)
    test_prediction(2, W1, b1, W2, b2, W3, b3)
    test_prediction(3, W1, b1, W2, b2, W3, b3)

In [6]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2, W3, b3)
get_accuracy(dev_predictions, Y_dev)

[7 4 0 3 3 2 0 1 2 9 9 5 3 4 4 1 0 6 7 2 5 9 0 4 7 6 7 8 6 6 6 0 5 6 2 4 6
 2 9 6 0 5 7 6 3 0 1 8 6 7 6 1 9 4 5 4 9 3 3 3 2 3 4 6 1 2 6 3 4 0 6 7 6 8
 4 2 3 8 3 8 1 0 2 8 7 9 3 3 5 8 0 7 8 8 5 0 7 3 6 1 8 6 4 7 1 0 7 6 0 0 0
 1 3 1 2 3 6 2 5 0 3 1 1 1 2 9 3 7 5 5 0 1 1 4 6 6 0 6 6 5 0 4 1 9 5 0 3 3
 2 4 8 1 1 6 6 3 9 0 7 1 9 1 1 3 6 9 4 4 8 4 8 7 0 5 3 9 1 9 4 2 5 0 1 8 1
 9 5 4 0 5 8 5 7 1 0 9 7 3 6 7 2 6 0 1 9 9 9 9 6 5 2 0 5 2 0 6 0 2 9 8 2 3
 7 5 2 6 2 5 3 7 7 3 7 1 8 9 4 4 1 8 3 6 9 6 2 8 9 3 9 7 1 9 3 3 7 1 1 3 1
 2 1 4 2 4 0 7 6 9 7 3 1 8 1 1 5 2 8 7 8 3 7 1 7 2 2 0 1 7 7 5 6 4 0 3 8 1
 0 5 3 5 1 3 1 2 0 6 1 2 9 9 1 5 5 8 3 8 4 5 9 5 7 3 8 3 2 6 0 9 1 9 2 3 7
 3 4 2 8 7 1 8 7 1 9 7 6 2 8 2 3 2 6 2 4 6 0 8 1 7 3 5 6 4 7 4 3 9 4 9 0 1
 3 2 1 3 5 1 8 9 4 2 0 8 4 4 8 2 4 0 9 9 0 8 0 3 2 4 8 3 8 1 6 9 6 1 8 2 4
 9 8 7 2 4 5 3 9 7 5 6 1 1 7 9 5 7 5 6 9 4 7 0 3 0 8 9 6 2 0 8 9 7 3 0 8 6
 7 1 9 2 3 2 4 2 3 5 5 8 0 0 5 9 3 7 1 5 5 2 9 6 0 6 7 9 9 3 4 9 4 9 9 9 5
 8 7 3 2 7 3 8 1 0 9 5 2 

0.924